#### 스타벅스 서울시 매장 위치 크롤링

##### 스타벅스 웹 사이트
: https://www.starbucks.co.kr/index.do

In [1]:
# 셀레니움 등 라이브러리 임포트
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import pandas as pd
from tqdm.autonotebook import tqdm

C:\Users\Administrator\AppData\Local\Temp\ipykernel_11568\184023847.py:6: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [2]:
# 크롬 웹 드라이버 실행
driver = webdriver.Chrome()
url = 'https://www.starbucks.co.kr/store/store_map.do?disp=locale'
driver.get(url)

In [6]:
# 웹 드라이버 선택할 위치값 작성 (20년 8월과 동일)
btn_busan = '#container > div > form > fieldset > div > section > article.find_store_cont > ' + \
            'article > article:nth-child(4) > div.loca_step1 > div.loca_step1_cont > ul > ' + \
            'li:nth-child(6) > a'
# btn_busan

driver.find_element(By.CSS_SELECTOR, btn_busan).click()

In [7]:
# 부산이 선택된 상태
# '전체' 클릭
btn_all = '#mCSB_2_container > ul > li > a'
driver.find_element(By.CSS_SELECTOR, btn_all).click()

In [8]:
# 스타벅스는 한 화면에 전부 리스트업되기 때문에 함수를 만들 필요 없음
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')
# soup

In [9]:
# 검색 결과 위치 가져오기
starbucks_soup_list = soup.select('li.quickResultLstCon')
len(starbucks_soup_list)

137

In [10]:
starbucks_soup_list[0]

<li class="quickResultLstCon" data-code="3705" data-hlytag="null" data-index="0" data-lat="35.262466" data-long="129.091353" data-name="금정남산동DT" data-storecd="1605" style="background:#fff"> <strong data-my_siren_order_store_yn="N" data-name="금정남산동DT" data-store="1605" data-yn="N">금정남산동DT  </strong> <p class="result_details">부산광역시 금정구 중앙대로 1989 (남산동)<br/>1522-3232</p> <i class="pin_generalDT">리저브 매장 2번</i></li>

In [11]:
# 매장 정보 확인
starbucks_store = starbucks_soup_list[0]
name = starbucks_store.select('strong')[0].text.strip()
lat = starbucks_store['data-lat'].strip()
lng = starbucks_store['data-long'].strip()
store_type = starbucks_store.select('i')[0]['class'][0].split('_')[1] # _로 자르면 두번째 값이 general/reserve
address = starbucks_store.select('p')[0].get_text(separator='|').split('|')[0]

print(name) # 매장명
print(lat, lng, sep=' / ') # 위도, 경도 (매장 지도 위치)
print(store_type)
print(address)

금정남산동DT
35.262466 / 129.091353
generalDT
부산광역시 금정구 중앙대로 1989 (남산동)


In [12]:
# 137개 매장 데이터 가져오기
starbucks_seoul_list = []

for item in tqdm(starbucks_soup_list):
    name = item.select('strong')[0].text.strip()
    lat = item['data-lat'].strip()
    lng = item['data-long'].strip()
    store_type = item.select('i')[0]['class'][0].split('_')[1] # _로 자르면 두번째 값이 general/reserve
    address = item.select('p')[0].get_text(separator='|').split('|')[0]

    starbucks_seoul_list.append([name, lat, lng, store_type, address])

100%|██████████| 137/137 [00:00<00:00, 6854.01it/s]


In [13]:
len(starbucks_seoul_list)

137

In [14]:
# DataFrame 생성
columns = ['매장명', '위도', '경도', '매장 타입', '주소']
df = pd.DataFrame(starbucks_seoul_list, columns=columns)
df

,매장명,위도,경도,매장 타입,주소
0,금정남산동DT,35.262466,129.091353,generalDT,부산광역시 금정구 중앙대로 1989 (남산동)
1,금정구청DT,35.238784,129.092964,generalDT,부산광역시 금정구 중앙대로 1730 (부곡동)
2,온천장역DT,35.224006,129.08656,generalDT,부산광역시 금정구 식물원로 13 (장전동)
3,부산대정문,35.232107,129.084565,general,부산광역시 금정구 부산대학로64번길 12 (장전동)
4,부산대역,35.230722,129.088165,general,"부산광역시 금정구 장전로12번길 64, 1~4층 (장전동)"
...,...,...,...,...,...
132,해운대 엑스더스카이,35.1599648,129.1697564,general,부산광역시 해운대구 달맞이길30
133,센텀신세계3F,35.16906348,129.12906568,general,부산광역시 해운대구 센텀남대로 35 (우동)
134,해운대중동역,35.1671846,129.1678855,general,부산광역시 해운대구 좌동순환로 6 (중동)
135,센텀몰1F,35.16981023,129.12778297,general,부산광역시 해운대구 센텀4로 15 (우동)


In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 137 entries, 0 to 136
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   매장명     137 non-null    object
 1   위도      137 non-null    object
 2   경도      137 non-null    object
 3   매장 타입   137 non-null    object
 4   주소      137 non-null    object
dtypes: object(5)
memory usage: 5.5+ KB


In [16]:
# 엑셀로 저장
df.to_excel('./busan_starbucks.xlsx', index=False)